<a href="https://colab.research.google.com/github/shelleyg-bit/canada-land-cover-classifier/blob/main/gee_download_images_v0_b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Dependencies 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install geopandas
!pip install geojson
!pip install geemap
!pip install geotable

In [3]:
import ee
ee.Authenticate() # give this notebook access to your google ee account
ee.Initialize() # this talks to ee backend to find out about all features of ee

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=vBa-gx59cd045vuvb3lba8DUfixA6fV_m7Gih0AcGPQ&tc=svw5ccBMIAiuA5HdcTzbEvhjAQeGI_Sc7p4YWZvs5FU&cc=F-siATT--k7bEGiQ464t1MsN5qMmJRFdTpaUcM6xxZ8

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWhsOnJebtgpQy4DCEUMnxsnGwjz2LxSErHVB4f2edqeqGyxkrfxIwU

Successfully saved authorization token.


In [4]:
import sys
sys.path.append('/content/drive/MyDrive/FarmHand/data_prep')

In [5]:
path1 ='/content/drive/MyDrive/FarmHand/data/All MB kml files/original_kmz_files/'
path2 ='/content/drive/MyDrive/farmhand/Data/farms_sheet.xlsx'
sheet ='Farm data (for farms created in'
path3 ='/content/drive/MyDrive/farmhand/Data/Mint farm data (2022 season)/Farmer data of mint. (1).xlsx'

In [6]:
from Kmz_n_excel_comb_2shp import comb_kmz_n_excel_2shp

df = comb_kmz_n_excel_2shp(path1, path2, sheet, path3)

shape file stored at the following location /content/drive/MyDrive/farmhand/Data/Mint farm data (2022 season) 


In [7]:
df.head()

,Name,geometry,class
0,K Shankar guru,"POLYGON Z ((76.73941 15.24347 0.00000, 76.7394...",Chilli-Non Farmhand
1,siddappa,"POLYGON Z ((76.71973 15.26482 0.00000, 76.7195...",Chilli-Non Farmhand
2,chandru nayaka,"POLYGON Z ((76.96205 15.12738 0.00000, 76.9616...",Chilli-Non Farmhand
3,Dodda Basava G,"POLYGON Z ((76.72902 15.24342 0.00000, 76.7290...",Chilli-Non Farmhand
4,udaya m,"POLYGON Z ((76.96406 15.12368 0.00000, 76.9636...",Chilli-Non Farmhand


In [8]:
from shapely.geometry import Polygon

# Remove the z value from each point in polygon
for i, boundary in enumerate(df['geometry'].values):
  coords = list(boundary.exterior.coords)
  df['geometry'][i] = Polygon([xyz[0:2] for xyz in coords])

  

In [9]:
df.head()

,Name,geometry,class
0,K Shankar guru,"POLYGON ((76.73941 15.24347, 76.73942 15.24347...",Chilli-Non Farmhand
1,siddappa,"POLYGON ((76.71973 15.26482, 76.71954 15.26474...",Chilli-Non Farmhand
2,chandru nayaka,"POLYGON ((76.96205 15.12738, 76.96169 15.12735...",Chilli-Non Farmhand
3,Dodda Basava G,"POLYGON ((76.72902 15.24342, 76.72901 15.24344...",Chilli-Non Farmhand
4,udaya m,"POLYGON ((76.96406 15.12368, 76.96365 15.12385...",Chilli-Non Farmhand


In [10]:
df.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [12]:
import geemap
import geopandas as gpd

import numpy as np

import matplotlib.pyplot as plt


## Create an EE Geometry Object for Area of Interest 



In [13]:
#Carefully choosen post analyzing
bbox1 =[76.65, 15.2,76.8, 15.37 ] #chili farms aoi_1
bbox2 =[76.9, 15.05, 77.05, 15.15] #chili farms aoi_2
bbox3 =[79.1996545, 28.4018788, 89.6, 28.75 ] #mint farms aoi_3

In [146]:
from shapely.geometry import box, Polygon, MultiPolygon

chili_bbox1 = Polygon(box(*bbox1))
chili_bbox2 = Polygon(box(*bbox2))
mint_bbox = Polygon(box(*bbox3))
aoi_gdf = gpd.GeoDataFrame({'geometry': [chili_bbox1, chili_bbox2, mint_bbox],\
                            'areaid': ['chili_1', 'chili_2', 'mint' ]}, crs=df.crs)
all_aoi = geemap.geopandas_to_ee(aoi_gdf)

In [147]:
aoi_gdf

,geometry,areaid
0,"POLYGON ((76.80000 15.20000, 76.80000 15.37000...",chili_1
1,"POLYGON ((77.05000 15.05000, 77.05000 15.15000...",chili_2
2,"POLYGON ((89.60000 28.40188, 89.60000 28.75000...",mint


In [148]:
all_aoi.getInfo()

{'columns': {'areaid': 'String', 'system:index': 'String'},
 'features': [{'geometry': {'coordinates': [[[76.65, 15.2],
      [76.8, 15.2],
      [76.8, 15.37],
      [76.65, 15.37],
      [76.65, 15.2]]],
    'type': 'Polygon'},
   'id': '0',
   'properties': {'areaid': 'chili_1'},
   'type': 'Feature'},
  {'geometry': {'coordinates': [[[76.9, 15.05],
      [77.05, 15.05],
      [77.05, 15.15],
      [76.9, 15.15],
      [76.9, 15.05]]],
    'type': 'Polygon'},
   'id': '1',
   'properties': {'areaid': 'chili_2'},
   'type': 'Feature'},
  {'geometry': {'coordinates': [[[79.1996545, 28.4018788],
      [89.6, 28.4018788],
      [89.6, 28.75],
      [79.1996545, 28.75],
      [79.1996545, 28.4018788]]],
    'type': 'Polygon'},
   'id': '2',
   'properties': {'areaid': 'mint'},
   'type': 'Feature'}],
 'type': 'FeatureCollection'}

In [113]:
type(all_aoi.first())

ee.element.Element

In [128]:
f = ee.Feature(all_aoi.first())
f.getString('areaid')
f.id().getInfo()


'0'

In [83]:
aoi_geom = ee.Geometry(all_aoi.first().getInfo()['geometry'])

In [84]:
aoi_geom.getInfo()

{'coordinates': [[[76.65, 15.2],
   [76.8, 15.2],
   [76.8, 15.37],
   [76.65, 15.37],
   [76.65, 15.2]]],
 'type': 'Polygon'}


## Get Image Collection from Sentinel

- For each harvest cycle(Sept-Feb) from 2019-2022, filtering out only harvest cycle months and discarding non-usable months (May-June)
- Collecting all the remaining raw bands on sentinel as @Mani Babu suggested. (right now I just collected 10m bands (RGB+NearIR) & NDVI)
- Combining All the bands for each harvest cycle in a single Tiff file.
- Monthly mean of every band
- A single tif will contain MonthName_Band_No as one band , so total number of bands will be  in one tiff will be Number of Months* Number of bands 
- Resulting in One GeoTiff file per harvest cycle
- Adding Mint farms to the collection set.

In [26]:
# method to remove cloud from the image
def maskclouds(image):
    """To mask clouds using the Sentinel-2 QA band
    @param {ee.Image} image Sentinel-2 image
    @return {ee.Image} cloud masked Sentinel-2 image
    """
    band_qa = image.select('QA60')
    cloud_mask = ee.Number(2).pow(10).int()
    cirrus_mask = ee.Number(2).pow(11).int()
    mask = band_qa.bitwiseAnd(cloud_mask).eq(0) and(
        band_qa.bitwiseAnd(cirrus_mask).eq(0))
    return image.updateMask(mask).divide(10000)

def fndvi(image):
    """Add NDVI to each pixel in an image"""
    #TODO: ask toby about this NDVI calculation
    # since its 20m resolution
    # there are other NDVIs at 10m resolution
    ndvi = image.expression(
    "(NIR-RED)/(NIR+RED)",
    {
        'RED': image.select('B4').multiply(0.0001),
        'NIR' : image.select('B5').multiply(0.0001)
    
    });# okay
    ndf = ndvi.rename('FNDVI')
    results = ndf.copyProperties(image, ['system:time_start'])
    return image.addBands(results)

In [149]:
def get_composites_for_aoi(aoi):
  aoi_geom = ee.Geometry(aoi.getInfo()['geometry'])

  collection_sentinel = ('COPERNICUS/S2_SR')
  time_range = ee.DateRange('2019-08-01', '2020-05-31')
  start_date = time_range.start()
  end_date = time_range.end()
  count = end_date.difference(start_date, 'month').round().toInt()
  month_seq = ee.List.sequence(0, count.subtract(1))
  
  images_collection = ee.ImageCollection(collection_sentinel)
  
  projection = images_collection.first().select('B2').projection()
  images_collection_aoi = images_collection.filterBounds(aoi_geom)
  # return images_collection_aoi.filterDate(start_date, end_date).size().getInfo() # for testing

  def create_monthly_max_composite(month_no):
    month_start_date = start_date.advance(month_no, 'month')
    month_end_date = month_start_date.advance(1, 'month')
    duration = ee.DateRange(month_start_date, month_end_date)
  
    def temporal_composite(duration, reducer):
      ## FIXME: filter out months with no data
      ## some months with many cloudy days
      ## might end up with No data for the aoi
      return (images_collection_aoi.
              filterDate(duration.start(), duration.end()).
              filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)).
              map(maskclouds).
              select(['B8', 'B4', 'B3', 'B2', 'B5']).
              reduce(reducer).
              reproject(projection).
              clip(aoi_geom))

    image = (temporal_composite(duration, ee.Reducer.max()).
            float().
            set('system:time_start', month_start_date.millis()).
            set('system:id', month_start_date.format('YYYYMM')).
            set('system:index', month_start_date.format('YYYYMM')))
  
    def new_name(old_name):
      return month_name.cat(old_name)
    month_name = month_start_date.format("YYYY'_'MMM'_'")
    band_names = image.bandNames()
    new_band_names = band_names.map(new_name)
    image = image.rename(new_band_names)
    return image
  
  # return create_monthly_max_composite(4) # for testing
  
  monthly_collection = ee.ImageCollection.fromImages(month_seq.map(create_monthly_max_composite))

  aoi_f = ee.Feature(aoi)
  out_dir=f"FarmHand/data/sentinel_composites/{aoi.getString('areaid')}"
  geemap.ee_export_image_collection_to_drive(monthly_collection, descriptions=None, folder=out_dir, region=aoi_geom)
  return monthly_collection


In [139]:
aoi_list = all_aoi.toList(3)

In [150]:
aoi1 = ee.Feature(aoi_list.get(0).getInfo())
get_composites_for_aoi(aoi1)

Total number of images: 10

Exporting 201908 ...
Exporting 201909 ...
Exporting 201910 ...
Exporting 201911 ...
Exporting 201912 ...
Exporting 202001 ...
Exporting 202002 ...
Exporting 202003 ...
Exporting 202004 ...
Exporting 202005 ...


In [151]:
aoi2 = ee.Feature(aoi_list.get(1).getInfo())
get_composites_for_aoi(aoi2)

Total number of images: 10

Exporting 201908 ...
Exporting 201909 ...
Exporting 201910 ...
Exporting 201911 ...
Exporting 201912 ...
Exporting 202001 ...
Exporting 202002 ...
Exporting 202003 ...
Exporting 202004 ...
Exporting 202005 ...


In [152]:
aoi3 = ee.Feature(aoi_list.get(2).getInfo())
get_composites_for_aoi(aoi3)

Total number of images: 10

Exporting 201908 ...
Exporting 201909 ...
Exporting 201910 ...
Exporting 201911 ...
Exporting 201912 ...
Exporting 202001 ...
Exporting 202002 ...
Exporting 202003 ...
Exporting 202004 ...
Exporting 202005 ...


In [145]:
aoi3.getInfo()

{'geometry': {'coordinates': [[[79.1996545, 28.4018788],
    [89.6, 28.4018788],
    [89.6, 28.75],
    [79.1996545, 28.75],
    [79.1996545, 28.4018788]]],
  'type': 'Polygon'},
 'id': '2',
 'properties': {'id': 'mint'},
 'type': 'Feature'}

In [107]:
aoi.getInfo()

{'geometry': {'coordinates': [[[76.65, 15.2],
    [76.8, 15.2],
    [76.8, 15.37],
    [76.65, 15.37],
    [76.65, 15.2]]],
  'type': 'Polygon'},
 'id': '0',
 'properties': {'areaid': 'chili_1'},
 'type': 'Feature'}

In [68]:
monthly_collection = get_composites_for_aoi(aoi.first())



In [78]:
monthly_collection.first().bandNames().getInfo()

[]

In [69]:
monthly_collection.getInfo()

{'bands': [],
 'features': [{'bands': [],
   'id': '201908',
   'properties': {'system:footprint': None,
    'system:index': '201908',
    'system:time_start': 1564617600000},
   'type': 'Image'},
  {'bands': [],
   'id': '201909',
   'properties': {'system:footprint': None,
    'system:index': '201909',
    'system:time_start': 1567296000000},
   'type': 'Image'},
  {'bands': [],
   'id': '201910',
   'properties': {'system:footprint': None,
    'system:index': '201910',
    'system:time_start': 1569888000000},
   'type': 'Image'},
  {'bands': [{'crs': 'EPSG:32635',
     'crs_transform': [10, 0, 499980, 0, -10, 3000000],
     'data_type': {'precision': 'float', 'type': 'PixelType'},
     'dimensions': [3098, 3364],
     'id': '2019_Nov_B8_max',
     'origin': [599503, 44436]},
    {'crs': 'EPSG:32635',
     'crs_transform': [10, 0, 499980, 0, -10, 3000000],
     'data_type': {'precision': 'float', 'type': 'PixelType'},
     'dimensions': [3098, 3364],
     'id': '2019_Nov_B4_max',
   

In [70]:
monthly_collection.aggregate_array('crs').getInfo()

[]

In [71]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [74]:

out_dir=f"/content/drive/MyDrive/FarmHand/data/sentinel_composites/{aoi.get('areaid').getInfo()}"
#geemap.ee_export_image(ee.Image(monthly_collection.toList(10).get(3)), filename=f"{out_dir}/test.tif", region=aoi_geom)
geemap.ee_export_image_to_drive(ee.Image(monthly_collection.toList(10).get(3)), 'export_image', folder=out_dir, region=aoi_geom)

Exporting export_image ...


# Visualizing Satellite Imagery

[Indices & composities](https://github.com/sentinel-hub/custom-scripts/tree/master/sentinel-2/false_color_infrared)

[Gee guide for visualization parameters](https://tutorials.geemap.org/Image/image_visualization/)

In [ ]:

Map = geemap.Map(location=(15.3510, 76.7318), zoom=10)
vis_params = {
    'bands': ['B11_max', 'B8_max', 'B2_max'],
    'min': 0.2,
    'max': 1.0
}
vis_type = 'agriculture(B11,B8,B2)'

#image = ee.Image(images_collection.toList(100).get(10)) # map one image from collection
Map.addLayer(month1.clip(aoi), vis_params, vis_type) 
Map.addLayer(aoi, {'color': 'green'}, 'AOI', opacity=0.5)

# prepare farms polygons for plotting
#farms_gdf = gpd.GeoDataFrame({'geometry': df['geometry'].values}, crs=df.crs)
farm_geom = geemap.geopandas_to_ee(df)

Map.addLayer(farm_geom, {'color': 'cyan'}, opacity=0.5)
Map


Map(center=[15.351, 76.7318], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…